# Evaluate Fisher information

Johann Brehmer, Kyle Cranmer, Marco Farina, Felix Kling, Duccio Pappadopulo, Josh Ruderman 2018

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import logging
import os

import madminer.core
from madminer.fisherinformation import FisherInformation, profile_information, project_information
from madminer.plotting import plot_fisher_information_contours_2d


In [2]:
logging.basicConfig(
    format='%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s',
    datefmt='%H:%M',
    level=logging.INFO
)

for key in logging.Logger.manager.loggerDict:
    if "madminer" not in key:
        logging.getLogger(key).setLevel(logging.WARNING)

In [3]:
base_dir = '/Users/johannbrehmer/work/projects/madminer/diboson_mining/'
mg_dir = '/Users/johannbrehmer/work/projects/madminer/MG5_aMC_v2_6_4/'

In [4]:
sample_dir = base_dir + 'data/samples/wgamma_sys/'
card_dir = base_dir + 'cards/wgamma/'
ufo_model_dir = card_dir + 'SMWgamma_UFO'
run_card_dir = card_dir + 'run_cards/'
mg_process_dir = base_dir + 'data/mg_processes/wgamma_sys/'
log_dir = base_dir + 'logs/wgamma/'
temp_dir = base_dir + 'data/temp'
delphes_dir = mg_dir + 'Delphes'
model_dir = base_dir + 'data/models/wgamma_sys/'
result_dir = base_dir + 'data/results/wgamma_sys/'

## Main functions

In [13]:
fisher_all = FisherInformation(sample_dir + 'samples.h5')
fisher_tight = FisherInformation(sample_dir + 'samples_tight.h5')

19:48 madminer.fisherinfor INFO    Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/data/samples/wgamma_sys/samples.h5
19:48 madminer.fisherinfor INFO    Found 2 parameters
19:48 madminer.fisherinfor INFO    Found 55 nuisance parameters
19:48 madminer.fisherinfor INFO    Found 103 benchmarks, of which 6 physical
19:48 madminer.fisherinfor INFO    Found 33 observables: et_miss, phi_miss, e_visible, eta_visible, e_l1, pt_l1, eta_l1, phi_l1, e_a1, pt_a1, eta_a1, phi_a1, e_j1, pt_j1, eta_j1, phi_j1, pdgid_l1, m_la, m_lmet, m_amet, pt_la, pt_lmet, pt_amet, deltaphi_la, deltaphi_lmet, deltaphi_amet, deltaeta_la, m_almet, pt_almet, mt, phi_minus, phi_plus, phi
19:48 madminer.fisherinfor INFO    Found 1764309 events
19:48 madminer.fisherinfor INFO    Found morphing setup with 6 components
19:48 madminer.fisherinfor INFO    Found nuisance morphing setup
19:48 madminer.fisherinfor INFO    Loading data from /Users/johannbrehmer/work/projects/madminer/diboson_mining/dat

## Main function

In [14]:
def calculate_info(filename, mode, tight_cuts=False, **kwargs):
    fisher_ = fisher_tight if tight_cuts else fisher_all
    ending_ = '_tight.npy' if fisher_tight else '.npy'
    
    if mode == "truth":
        fisher_info, cov = fisher_.calculate_fisher_information_full_truth(
            theta=[0.,0.],
            luminosity = 300000.,
            **kwargs
        )
    elif mode == "rate":
        fisher_info, cov = fisher_all.calculate_fisher_information_rate(
            theta=[0.,0.],
            luminosity = 300000.,
            **kwargs
        )
    elif mode == "histo":
        fisher_info, cov = fisher_all.calculate_fisher_information_hist1d(
            theta=[0.,0.],
            luminosity = 300000.,
            **kwargs
        )
    elif mode == "ml":
        fisher_info, cov = fisher_all.calculate_fisher_information_full_detector(
            theta=[0.,0.],
            luminosity = 300000.,
            **kwargs
        )

    np.save(result_dir + 'information_{}{}'.format(filename, ending_), fisher_info)
    np.save(result_dir + 'covariance_{}{}'.format(filename, ending_), cov)

    fisher_info_nuisance = fisher_.calculate_fisher_information_nuisance_constraints()
    fisher_info, cov = profile_information(fisher_info + fisher_info_nuisance, [0,1], covariance=cov)
    
    np.save(result_dir + 'profiled_information_{}{}'.format(filename, ending_), fisher_info)
    np.save(result_dir + 'profiled_covariance_{}{}'.format(filename, ending_), cov)


## Truth-level info

In [ ]:
calculate_info('parton', 'truth', False)
calculate_info('parton', 'truth', True)

## Rate-only info

In [ ]:
calculate_info('rate', 'xsec', False)
calculate_info('rate', 'xsec', True)

## Full Fisher info (ML)

In [ ]:
calculate_info('full', 'ml', False, model_file=model_dir + "sally_ensemble_all", mode="score")
calculate_info('full', 'ml', True, model_file=model_dir + "sally_ensemble_all_tight", mode="score")

In [ ]:
calculate_info('full_infomode', 'ml', False, model_file=model_dir + "sally_ensemble_all", mode="information")
calculate_info('full_infomode', 'ml', True, model_file=model_dir + "sally_ensemble_all_tight", mode="information")

In [ ]:
calculate_info('full_shuffled', 'ml', False, mode="score")

In [ ]:
calculate_info('minimal', 'ml', False, model_file=model_dir + "sally_ensemble_minimal", mode="score")
calculate_info('minimal', 'ml', True, model_file=model_dir + "sally_ensemble_minimal_tight", mode="score")

In [ ]:
calculate_info('phi', 'ml', True, model_file=model_dir + "sally_ensemble_phi_tight", mode="score")

## Histogram info

In [ ]:
filenames = ['phi', 'met', 'ptl', 'pta', 'deltaphi_lv', 'deltaphi_la']
observables = ['phi', 'et_miss', 'pt_l1', 'pt_a1', 'deltaphi_lmet', 'deltaphi_la']
bins = 25

for filename, obs in zip(filenames, observables):
    calculate_info('histo_' + filename, 'histo', True, observable=obs, nbins=bins, histrange=None)
    